# Hotspot Transit Improvements

## Criteria
* transit routes that do not run on the SHN
* average route speed is above **12 mph**  **AND**
* 50%+ of its trips are competitive against a car traveling the same path (competitive is 1.5x...so bus can take 50% more time)

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import branca 
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils
from bus_service_utils import better_bus_utils

catalog = intake.open_catalog("../bus_service_increase/*.yml")

CUSTOM_CATEGORICAL = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
    cp.CALITP_CATEGORY_BRIGHT_COLORS[5], # purple  
    cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
]

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

In [ ]:
speed_dict = {"mean_speed_mph": 12}
trip_dict = {"pct_trips_competitive": 0.50}

gdf = better_bus_utils.select_transit_routes_hotspot_improvements(
    speed_dict = speed_dict,
    trip_dict = trip_dict
)

In [ ]:
def subset_to_district(gdf: gpd.GeoDataFrame, district: str) -> gpd.GeoDataFrame: 
    keep_cols = [
        "calitp_itp_id", "route_id", 
        "mean_speed_mph", "pct_trips_competitive",
        "category", "geometry"]
    
    df = (gdf[gdf.caltrans_district == district]
          .reset_index(drop=True)
          [keep_cols]
         )
    
    df2 = df.assign(
        mean_speed_mph = df.mean_speed_mph.round(2),
        pct_trips_competitive = df.pct_trips_competitive.round(3)
    )
               
    return df2

In [ ]:
TILES = "CartoDB Positron" 

def make_map(gdf: gpd.GeoDataFrame, district: str):        
    m = gdf.explore("mean_speed_mph", categorical = False, 
                    cmap = ZERO_THIRTY_COLORSCALE, tiles = TILES)
    
    display(m)
    
    m2 = gdf.explore("route_id", categorical = True, 
                    cmap = "tab20", tiles = TILES)
    
    display(m2)
    
    table = gdf.sort_values(["mean_speed_mph", "pct_trips_competitive"], 
                           ascending=[True, True]
                           ).drop(columns = ["geometry", "category"])
    
    
    rename_cols = {
        "calitp_itp_id": "ITP ID", 
        "route_id": "Route ID",
        "mean_speed_mph": "Avg Speed (mph)", 
        "pct_trips_competitive": "% trips competitive",
    }
    
    table = table.rename(columns = rename_cols)
    
    df_style = (table.style
                .format(subset=["Avg Speed (mph)"], 
                                **{'formatter': '{:,.1f}'})
                .format(subset=["% trips competitive"],
                        **{'formatter': '{:,.2}'}
                ).set_properties(subset=['ITP ID', 'Route ID'], 
                                **{'text-align': 'left'})
                .set_properties(subset=['Avg Speed (mph)', '% trips competitive'], 
                               **{'text-align': 'center'})
                .set_table_styles([dict(selector='th', 
                                        props=[('text-align', 'center')])
                                        ])
           .hide(axis="index")
           .to_html()
          )
    
    display(HTML(df_style))
   

## District Maps

In [ ]:
districts = gdf[gdf.caltrans_district.notna()].caltrans_district.unique().tolist()

for d in sorted(districts):
    display(HTML(f"<h3> District {d}</h3>"))

    subset = subset_to_district(gdf, d)

    if len(subset) > 0:
        make_map(subset, d)
    else:
        print("No routes meet this criteria")